In [17]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# urlの取得
def url_to_soup(url):
    req = requests.get(url)
    return BeautifulSoup(req.content, 'html.parser')

def result_data(url):
    soup = url_to_soup(url)

    # 土の状態
    condition = soup.find(id="race-data02").get_text().replace('\n','').split(';')[1].split('　')[2][0:1]

    # レースの長さ
    race_len = int(soup.find(id="race-data01-a").get_text().replace('\n','').split('　')[3].replace(',','')[1:5])

#     # 1位
#     hukusyo_list = []
#     hukusyo_list.append(int(p.sub("", str(soup.find_all('tr', class_='bg-1chaku')[0]).split('</td>')[2]).replace('\n','') ))

    # レース日
    race_date = soup.find(id="race-data01-a").get_text().replace('\n','').split(';')[0].split('第')[0]
    return condition, race_len, race_date

re_da = list(result_data("https://www.nankankeiba.com/race_info/2019120620140512.do"))



In [18]:
def get_data(num):
        # csvファイルの読み込み
    # 欠損地を含む行を削除
    # indexの振り直し
    a = pd.read_csv(str(num) + ".csv", index_col=0).dropna(how='any', axis=0)
    a = a.reset_index(drop=True)
    # a = a[0:10].reset_index(drop=True)

    # 競馬場の判定
    for i in range(len(a)):
        if "大井" in a.loc[i, '競馬場']:
            a.loc[i, '競馬場'] = 1
        else:
            a.loc[i, '競馬場'] = 0

    # 大井競馬場であるレースだけを抽出
    a = a[a['競馬場'] == 1].drop('競馬場', axis=1)
    a = a.reset_index(drop=True)

    #問題解決URL
    # http://hirotaka-hachiya.hatenablog.com/entry/2016/01/04/171349
    # 列を特定してから行を特定しているため二段階になり時間がかかってしまう

    # 距離の判定
    for i in range(len(a)):
        if a.loc[i,'距離'] == re_da[1]:
            a.loc[i, '距離'] = 0
        else:
            a.loc[i, '距離'] = int(a.loc[i, '距離']) - 1400

    # 着順変換
    for i in range(len(a)):
        a.loc[i, '着順'] = a.loc[i, '着順'].split("/")[0]

    # タイムを秒数に変換
    from datetime import datetime as dt
    for i in range(len(a)):
        time = dt.strptime(a.loc[i, 'タイム'] , '%M:%S.%f') - dt(1900, 1,1)
        time = time.total_seconds()
        a.loc[i, 'タイム'] = str(time)
    #     a.loc[i, 'タイム'] = time.strftime('%M:%S.%f')

    # 天候馬場のデータを整形
    for i in range(len(a)):
        if '/重' == a.loc[i, '天候馬場'][-2:]:
            a.loc[i, 'soil_heavy'] = 1
        else:
            a.loc[i, 'soil_heavy'] = 0
    for i in range(len(a)):
        if '稍重' == a.loc[i, '天候馬場'][-2:]:
            a.loc[i, 'soil_s_heavy'] = 1
        else:
            a.loc[i, 'soil_s_heavy'] = 0
    for i in range(len(a)):
        if '/良' == a.loc[i, '天候馬場'][-2:]:
            a.loc[i, 'soil_good'] = 1
        else:
            a.loc[i, 'soil_good'] = 0 
    for i in range(len(a)):
        if '不良' == a.loc[i, '天候馬場'][-2:]:
            a.loc[i, 'soil_bad'] = 1
        else:
            a.loc[i, 'soil_bad'] = 0 
    # 天候馬場の列を削除
    a = a.drop('天候馬場', axis=1)
    
#     a.to_csv('a' + str(num) + '.csv')

In [19]:
for i in range(15):
    get_data(i+1)

FileNotFoundError: [Errno 2] File b'1.csv' does not exist: b'1.csv'